# setup

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=2dad364ba357e53e79887adc01fb82842d69c90f176412c83dac020bd6942ae3
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/COEN242/dataset_updated.zip","r") as zip_ref:
    zip_ref.extractall("/content/datasets")

# code

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import time
import psutil
def QuestionOne(exemem='10g',drmem='2g',resultsize='3g'):
  spark = SparkSession.builder\
      .config('spark.executor.memory', exemem)\
      .config('spark.driver.memory', drmem)\
      .config('spark.driver.maxResultSize', resultsize)\
      .master("local[*]")\
      .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')\
      .config('spark.sql.shuffle.partitions', '4') \
      .getOrCreate()

  #list of stopwords
  stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", 
              "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", 
              "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", 
              "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", 
              "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", 
              "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", 
              "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", 
              "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", 
              "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", 
              "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", 
              "s", "t", "can", "will", "just", "don", "should", "now"]

  df = spark.read.text("/content/datasets/dataset_updated/data_300MB.txt")  # Replace with your actual path
  df = df.filter(F.col("value") != "")  # Remove empty rows

  # DataFrame for all words
  word_counts_all = (
      df.withColumn("word", F.explode(F.split(F.col("value"), "\s+")))
      .withColumn("word", F.regexp_replace("word", "[^\w]", ""))
      .withColumn("word", F.lower(F.col("word")))
      .filter(~(F.col("word").isin(stopwords) | (F.trim(F.col("word")) == "")))  # Filter out stopwords and empty words 
      .groupBy("word")
      .count()
      .sort("count", ascending=False)
  )

  # DataFrame for words with more than 6 characters
  word_counts_6char = (
      word_counts_all.filter(F.length(F.col("word")) > 6)
  )

  # Top 100 for all words
  word_counts_all.show(100)

  # Top 100 for words with more than 6 characters
  word_counts_6char.show(100)

exememory = ['2g','4g','6g','8g','10g']
drememory = ['1g','2g']
resultsizememory = ['1g','2g','3g']

#QuestionOne()


results = []

for executor in exememory:
    for driver in drememory:
        for maxResultSize in resultsizememory:
            start_time = time.time()
            QuestionOne(executor, driver, maxResultSize)
            # Perform the desired operation here
            
            # Calculate the elapsed time
            elapsed_time = time.time() - start_time
            
            # Get system resource usage
            cpu_percent = psutil.cpu_percent()
            memory_percent = psutil.virtual_memory().percent
            
            # Create a dictionary to store the results
            result = {
                'ExecutorMemory': executor,
                'DriverMemory': driver,
                'MaxResultSize': maxResultSize,
                'ElapsedTime': elapsed_time,
                'CPUUsage': cpu_percent,
                'MemoryUsage': memory_percent
            }
            
            # Append the result to the list
            results.append(result)

# Create a Pandas DataFrame from the results list
df_results = pd.DataFrame(results)


Streaming output truncated to the last 5000 lines.
|         vote| 38215|
|        world| 38038|
|         part| 37846|
|        clear| 37839|
|      members| 37745|
|          may| 37144|
|        still| 36946|
|         year| 36770|
|   particular| 36430|
|          use| 36111|
|         know| 36081|
|          see| 35539|
|          mrs| 35100|
|       system| 34980|
|       energy| 34779|
+-------------+------+
only showing top 100 rows

+-------------+------+
|         word| count|
+-------------+------+
|     european|318567|
|   commission|172745|
|    president|146787|
|   parliament|122013|
|      council|107687|
|    countries| 93485|
|    important| 81677|
|    therefore| 68307|
|      support| 68001|
|      however| 65866|
|     economic| 60150|
|    committee| 59426|
|    political| 56393|
|      believe| 52836|
|  development| 52463|
| commissioner| 50938|
|     proposal| 50244|
|     question| 45842|
|    agreement| 45484|
|     citizens| 44331|
|    situation| 42968|
| 

In [ ]:
df_results

,ExecutorMemory,DriverMemory,MaxResultSize,ElapsedTime,CPUUsage,MemoryUsage
0,2g,1g,1g,139.529236,89.4,15.9
1,2g,1g,2g,103.626339,92.7,16.3
2,2g,1g,3g,106.730660,94.4,16.4
3,2g,2g,1g,99.759879,93.2,16.5
4,2g,2g,2g,100.471095,92.1,16.5
5,2g,2g,3g,101.383979,93.4,14.7
6,4g,1g,1g,98.005978,91.5,14.6
7,4g,1g,2g,98.066363,93.2,15.1
8,4g,1g,3g,102.447779,93.6,15.0
9,4g,2g,1g,97.645176,91.4,15.0
